In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from bokeh.io import curdoc
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, HoverTool
from bokeh.models.widgets import PreText, Select
from bokeh.plotting import figure, show


In [2]:
try:
    from functools import lru_cache
except ImportError:
    print("WARNING: Cache for this example is available on Python 3 only.")
    def lru_cache():
        def dec(f):
            def _(*args, **kws):
                return f(*args, **kws)
            return
        return dec

In [3]:
# Create mock datetime data

datentime = pd.date_range('2014-9-13', periods=2000, freq='15min')
#datentime

In [4]:
# Create mock data – periodic data with white noise: y1, y2, y3

x = np.linspace(0,200,2000)
amp1, freq1 = 1, 0.2
y1_signal = amp1*np.sin(2*np.pi*freq1*x) + 5
y1 = y1_signal + np.random.normal(0, np.sqrt(amp1), 2000)

amp2, freq2 = 3, 0.1
y2_signal = amp2*np.sin(2*np.pi*freq2*x) + 10
y2 = y2_signal + np.random.normal(0, np.sqrt(amp2), 2000)

amp3, freq3 = 2, 0.05
y3_signal = amp1*np.sin(2*np.pi*freq3*x) + 10
y3 = y3_signal + np.random.normal(0, np.sqrt(amp3), 2000)


In [5]:
# Put together fake data in a dataframe 'df'.

df = pd.DataFrame(columns=['datentime','Sample1', 'Sample2', 'Sample3'])
df['datentime'] = datentime
df["Sample1"] = y1
df["Sample2"] = y2
df["Sample3"] = y3
df

,datentime,Sample1,Sample2,Sample3
0,2014-09-13 00:00:00,3.181990,12.041410,9.310468
1,2014-09-13 00:15:00,6.456907,14.377553,8.902580
2,2014-09-13 00:30:00,7.291443,9.530672,10.715093
3,2014-09-13 00:45:00,6.725167,8.841953,10.795210
4,2014-09-13 01:00:00,6.551880,7.828228,11.385121
5,2014-09-13 01:15:00,5.696113,10.571480,9.524876
6,2014-09-13 01:30:00,5.796700,10.455005,8.765153
7,2014-09-13 01:45:00,6.117329,13.248391,9.666722
8,2014-09-13 02:00:00,6.475852,12.129620,11.700400
9,2014-09-13 02:15:00,5.880313,11.337890,12.120085


In [6]:
# Bokeh interface construction

# A list of fake sample 'gas species'
Species = ['Sample1', 'Sample2', 'Sample3']

def nix(val, lst):
    return [x for x in lst if x != val]

@lru_cache()
def load_ticker(ticker):
    return pd.DataFrame({ticker: df[ticker]})

@lru_cache()
def get_data(t1, t2):
    df1 = load_ticker(t1)
    df2 = load_ticker(t2)
    df3 = load_ticker('datentime')
    data = pd.concat([df1, df2, df3], axis=1)
    data = data.dropna()
    data['t1'] = data[t1]
    data['t2'] = data[t2]
    return data

# set up widgets
interval = PreText(text='', width=400)
stats = PreText(text='', width=400)
ticker1 = Select(value="Sample1", options=nix('Sample2', Species))
ticker2 = Select(value="Sample2", options=nix('Sample1', Species))

# set up plots

# define sources (A static source that always displays and a dynamic source that displays on selection)
# Choose tools
source = ColumnDataSource(data=dict(index=[], t1=[], t2=[], datentime=[]))
source_static = ColumnDataSource(data=dict(index=[], t1=[], t2=[], datentime=[]))
corrtoollist = 'pan, box_zoom, box_select, lasso_select, save, reset'
toollist = 'pan, box_zoom, xzoom_in, xzoom_out, yzoom_in, yzoom_out, xbox_select, save, reset'
hovertool = HoverTool(
    tooltips=[
#        ('Date and Time','@datentime{%F %T}'),
        ('Concentration','$y'),
            ],
#     formatters={
#         'Date and Time': 'datetime',
#     },
)
corrhovertool = HoverTool(
    tooltips=[
#        ('Date and Time','@datentime{%F}'),
        ('x','$x'),
        ('y','$y'),
            ],
#     formatters={
#         'Date and Time': 'datetime',
#     }
    )

# set up the correlation plot
corr = figure(plot_width=400, plot_height=400, tools=corrtoollist)
corr.background_fill_color= '#fafafa' # or efefef 

corr.circle('t1', 't2', size=2, source=source, alpha=0.5,
            color='#3a5785',
            nonselection_color='#3a5785', selection_color='#e98043',
            nonselection_alpha=0.1, selection_alpha=0.5)
corr.xaxis.axis_label_text_font_style = 'bold'
corr.yaxis.axis_label_text_font_style = 'bold'

corr.add_tools(corrhovertool)


# set up two time series

ts1 = figure(plot_width=900, plot_height=300, tools=toollist, toolbar_location='above', 
             x_axis_type='datetime', active_drag='box_zoom', active_inspect=None)
ts1.background_fill_color= '#fafafa'

ts1.circle('datentime', 't1', size=2, source=source_static,
            color='#3a5785', alpha=0.5)
ts1.circle('datentime', 't1', size=2, source=source,
            color='#3a5785', alpha=0.5, 
            nonselection_color='#3a5785', selection_color='#e98043',
            nonselection_alpha=0.1, selection_alpha=0.5)

ts1.add_tools(hovertool)

ts1.xaxis.axis_label = 'Date and Time'
ts1.xaxis.axis_label_text_color = 'black'
ts1.xaxis.axis_label_text_font_style = 'bold'
ts1.xaxis.axis_label_standoff = 10
ts1.yaxis.axis_label = 'Conc'
ts1.yaxis.axis_label_text_color = 'black'
ts1.yaxis.axis_label_text_font_style = 'bold'
ts1.yaxis.axis_label_standoff = 10
ts1.xaxis.formatter = DatetimeTickFormatter(
        minutes='%m/%d %H:%M',
        hours='%m/%d %H:%M',
        days='%m/%d %H:%M',
        months='%m/%d %H:%M',
    )

ts2 = figure(plot_width=900, plot_height=300, tools=toollist, toolbar_location='above', 
             x_axis_type='datetime', active_drag='box_zoom', active_inspect=None)
ts2.x_range = ts1.x_range
ts2.background_fill_color= '#fafafa'

ts2.circle('datentime', 't2', size=2, source=source_static, 
           color='#3a5785', alpha=0.4)
ts2.circle('datentime', 't2', size=2, source=source, 
           color='#3a5785', alpha=0.4,
           nonselection_color='#3a5785', selection_color='#e98043',
           nonselection_alpha=0.1, selection_alpha=0.5)

ts2.add_tools(hovertool)

ts2.xaxis.axis_label = 'Date and Time'
ts2.xaxis.axis_label_text_color = 'black'
ts2.xaxis.axis_label_text_font_style = 'bold'
ts2.xaxis.axis_label_standoff = 10
ts2.yaxis.axis_label = 'Conc.'
ts2.yaxis.axis_label_text_color = 'black'
ts2.yaxis.axis_label_text_font_style = 'bold'
ts2.yaxis.axis_label_standoff = 10
ts2.xaxis.formatter = DatetimeTickFormatter(
        minutes='%m/%d %H:%M',
        hours='%m/%d %H:%M',
        days='%m/%d %H:%M',
        months='%m/%d %H:%M',
    )

# set up callbacks

def ticker1_change(attrname, old, new):
    ticker2.options = nix(new, Species)
    update()
    
def ticker2_change(attrname, old, new):
    ticker1.options = nix(new, Species)
    update()
    

# updates when changing samples    
def update(selected=None):
    t1, t2 = ticker1.value, ticker2.value
    
    dfile = get_data(t1, t2)
    data = dfile[['t1', 't2', 'datentime']]
    source.data = data
    source_static.data = data
    
    update_interval(dfile)
    update_stats(dfile, t1, t2)
    
    corr.title.text = '%s vs %s' % (t2, t1)
    corr.xaxis.axis_label = '%s' % t1
    corr.yaxis.axis_label = '%s' % t2
    
    ts1.title.text, ts2.title.text = '%s species over time in 2014' % t1, '%s species over time in 2014' % t2

def update_interval(data):
    interval.text = 'From %s To %s' % (data['datentime'].iloc[0], data['datentime'].iloc[-1])
    
def update_stats(data, t1, t2):
    stats.text = str(data[[t1, t2]].describe())
        
ticker1.on_change('value', ticker1_change)
ticker2.on_change('value', ticker2_change)

# updates on selection
def selection_change(attrname, old, new):
    t1, t2 = ticker1.value, ticker2.value
    data = get_data(t1, t2)
    selected = source.selected.indices
    if selected:
        data = data.iloc[selected, :]
    update_interval(data)
    update_stats(data, t1, t2)
    
source.selected.on_change('indices', selection_change)

# set up layout

widgets = column(ticker1, ticker2, interval, stats)
main_row = row(widgets, corr)
series = column(ts1, ts2)
layout = column(main_row, series)

# initialize

update()

curdoc().add_root(layout)
curdoc().title = 'project_Bokeh'
show(layout)

/anaconda3/lib/python3.6/site-packages/bokeh/plotting/helpers.py:689: UserWarning: ZoomInTool,ZoomOutTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))
You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

